# NLI : Dataset SNLI

In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import random

In [2]:
train_dataset = load_dataset("snli", split='train')
test_dataset = load_dataset("snli", split='test')
validation_dataset = load_dataset("snli", split='validation')

Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


In [3]:
from data_utils import PreProcessor
preprocessor = PreProcessor()

train_dataset_processed = preprocessor.transform(train_dataset)
test_dataset_processed = preprocessor.transform(test_dataset)
validation_dataset_processed = preprocessor.transform(validation_dataset)

Loading cached processed dataset at /Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-0d830334c09a9df2.arrow
Loading cached processed dataset at /Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-98e87cabcc380f45.arrow
Loading cached processed dataset at /Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-3580dd0209eae8f7.arrow


In [31]:
from model import TransformerTrainer
from transformers import AutoModelForSequenceClassification

# Create an instance of the TransformerTrainer class with a BERT model
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-roberta-base')
transformer_trainer = TransformerTrainer(model)

import random
train_data_sample = random.sample(train_dataset_processed, k=100)
validation_data_sample = random.sample(validation_dataset_processed, k=100)

transformer_trainer.fit(train_data_sample, validation_data_sample)


/Users/pepegarcia/Documents/GitHub/NLI/.env/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/13 [00:00<?, ?it/s]

{'train_runtime': 88.1547, 'train_samples_per_second': 1.134, 'train_steps_per_second': 0.147, 'train_loss': 2.3644469334528995, 'epoch': 1.0}


TransformerTrainer(model=RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inpla...
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
  )
  (classifier): RobertaClassificationHead(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=3, bias=True)
  )
))

In [34]:
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

class TransformerTrainer2(BaseEstimator, TransformerMixin):
    def __init__(self, model, train_batch_size=8, eval_batch_size=32, epochs=1, warmup_steps=0.1,
                 eval_steps=500, metric_for_best_model='eval_loss', output_dir='./results',
                 load_best_model_at_end=True, random_seed=42):
        self.model = model
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size
        self.epochs = epochs
        self.warmup_steps = warmup_steps
        self.eval_steps = eval_steps
        self.metric_for_best_model = metric_for_best_model
        self.output_dir = output_dir
        self.load_best_model_at_end = load_best_model_at_end
        self.random_seed = random_seed

    def fit(self, X, y):
        
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            evaluation_strategy='steps',
            eval_steps=self.eval_steps,
            load_best_model_at_end=self.load_best_model_at_end,
            per_device_train_batch_size=self.train_batch_size,
            per_device_eval_batch_size=self.eval_batch_size,
            num_train_epochs=self.epochs,
            metric_for_best_model=self.metric_for_best_model,
            warmup_steps=self.warmup_steps
        )

        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=X,
            eval_dataset=y,
            compute_metrics=lambda pred: {"accuracy": accuracy_score(
                pred.label_ids, pred.predictions.argmax(axis=1))}
        )

        self.trainer.train()
        self.trained_model = self.trainer.model
        return self
    
    def predict(self, X, y=None):
        if not hasattr(self, 'trained_model'):
            raise ValueError("The model has not been trained yet. Please call 'fit' first.")

        # Reuse the existing trainer with the trained model
        self.trainer.model = self.trained_model

        predictions = self.trainer.predict(X)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        if y is not None:
            accuracy = accuracy_score(y, predicted_labels)
            return predicted_labels, accuracy
        else:
            return predicted_labels



In [35]:
from transformers import RobertaForSequenceClassification

# Create an instance of the TransformerTrainer class with a BERT model
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-roberta-base')
transformer_trainer = TransformerTrainer2(model)

import random
train_data_sample = random.sample(train_dataset_processed, k=100)
validation_data_sample = random.sample(validation_dataset_processed, k=100)

transformer_trainer.fit(train_data_sample, validation_data_sample)


  0%|          | 0/13 [00:00<?, ?it/s]

{'train_runtime': 83.103, 'train_samples_per_second': 1.203, 'train_steps_per_second': 0.156, 'train_loss': 2.3644469334528995, 'epoch': 1.0}


TransformerTrainer2(model=RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inpl...
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
  )
  (classifier): RobertaClassificationHead(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=3, bias=True)
  )
))

In [36]:
# Call the predict method of the TransformerTrainer object on your test data
predicted_labels = transformer_trainer.predict(test_dataset_processed)

  0%|          | 0/307 [00:00<?, ?it/s]

In [38]:
# Calculate the accuracy score
accuracy = accuracy_score(predicted_labels, np.array(test_dataset.filter(lambda x: x['label']!=-1)['label']))
accuracy

Loading cached processed dataset at /Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-f18f93f50f3bd4e5.arrow


0.32797231270358307